# Introduction to convolutional networks
### Based on Chapter 5 from "Deep Learning with Python" by F. Chollet

In [ ]:
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical

###  Load MNIST data
- Reshape images and scale 8-bit pixel values between zero and one
- Categorically encode labels, for example: "2" -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# Load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

def prepare_images(images):
    return images.reshape((-1, 28, 28, 1)).astype('float32') / 255.0

# Reshape images and scale between zero and one
train_images = prepare_images(train_images)
test_images = prepare_images(test_images)

# Categorical encoding of class labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Define convolutional network and add dense network on top as classifier. 
In the convolutional network, learn patterns from 3x3 windows. First convolution layer outputs feature maps of size (26, 26, 32). In the first two dimensions, size decreases due to border effects (there are only 26 pixel positions where 3x3 windows can be placed along an axis with 28 pixels). The size of the output feature map in the third dimension is a configurable parameter corresponding to different "filters", analogous to color channels in RGB image.

In [ ]:
model = models.Sequential()

# Convolutional network

# In the first layers, learn patterns from 3x3 windows
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

# Classifier
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

### Check network and input data
No cross-validation yet to ensure things are properly set-up

In [ ]:
history = model.fit(train_images, train_labels, epochs = 5, batch_size = 64)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy %.4f' % test_acc)